### Edit to recursively compute precision

Recursively remove the score part of file names

In [4]:
import os

# Create a df with two columns, one named class, the other precision

def compute_precision(directory):
    """
    Recursively scans all folders and subfolders in a directory
    and renames files by removing the score prefix from filenames.
    """
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                # Identify the file named like classname_'validation'.txt
                # Once identified by the element 'validation' extract the first element 
                # of the file name and store it in df under class colum. 
                # then access the .txt file and read the column Eval and compute 
                # precision (counts of 1/ total counts). 
                # Write the computed precision under the column precision of the same row. 
                # Go to the next class folder.
print ("precision for all classes computed")


Renamed: /mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/model_0/random/Buff-fronted Owl/0.1024_72_M5_SN08_20220809_010000.WAV -> /mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/model_0/random/Buff-fronted Owl/72_M5_SN08_20220809_010000.WAV
Renamed: /mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/model_0/random/Buff-fronted Owl/0.1486_54_M41_SN06_20221004_200000.WAV -> /mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/model_0/random/Buff-fronted Owl/54_M41_SN06_20221004_200000.WAV
Renamed: /mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/model_0/random/Buff-fronted Owl/0.1891_39_M14_SN11_20220802_031000.WAV -> /mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/model_0/random/Buff-fronted Owl/39_M14_SN11_20220802_031000.WAV
Renamed: /mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/model_0/random/Buff-fronted Owl/0.1378_15_M37_SN05_20220823_21